In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import KFold

In [3]:
sys.path.append('../..')

In [30]:
from loaders import load_sabes
from segmentation import reach_segment_sabes
from gaussian_util import gaussian_DI, gaussian_DI_trialized
from subspaces import estimate_autocorrelation

In [9]:
indy_files = glob('/mnt/Secondary/data/sabes/indy*')
loco_files = glob('/mnt/Secondary/data/sabes/loco*')

### M1 --> behavior 

In [14]:
start_times = {'indy_20160426_01': 0,
               'indy_20160622_01':1700,
               'indy_20160624_03': 500,
               'indy_20160627_01': 0,
               'indy_20160630_01': 0,
               'indy_20160915_01': 0,
               'indy_20160921_01': 0,
               'indy_20160930_02': 0,
               'indy_20160930_05': 300,
               'indy_20161005_06': 0,
               'indy_20161006_02': 350,
               'indy_20161007_02': 950,
               'indy_20161011_03': 0,
               'indy_20161013_03': 0,
               'indy_20161014_04': 0,
               'indy_20161017_02': 0,
               'indy_20161024_03': 0,
               'indy_20161025_04': 0,
               'indy_20161026_03': 0,
               'indy_20161027_03': 500,
               'indy_20161206_02': 5500,
               'indy_20161207_02': 0,
               'indy_20161212_02': 0,
               'indy_20161220_02': 0,
               'indy_20170123_02': 0,
               'indy_20170124_01': 0,
               'indy_20170127_03': 0,
               'indy_20170131_02': 0,
               }

In [11]:
# Evaluate over the entire duration of the time series
win = 5
lags = [0, 3, 5, 10, 20]

results_list = []

for data_file in tqdm(indy_files):
    dat = load_sabes(data_file)
    y = np.squeeze(dat['spike_rates'])
    z = np.squeeze(dat['behavior'])

    for lag in lags:

        ylag = y[:-lag]
        zlag = z[lag:]

        # Y -> Z
        di_yz = gaussian_DI(ylag, zlag, win)

        ylag = y[lag:]
        zlag = z[:-lag]

        # Z -> y
        di_zy = gaussian_DI(zlag, ylag, win)

        idx_split = np.array_split(np.arange(y.shape[0]), 10)
        di_zy10 = np.zeros(10)
        di_yz10 = np.zeros(10)
        
        for i, idx in enumerate(idx_split):
            
            y_ = y[idx]
            z_ = z[idx]

            ylag = y_[:-lag]
            zlag = z_[lag:]

            di_yz10[i] = gaussian_DI(ylag, zlag, win)

            ylag = y_[lag:]
            zlag = z_[:-lag]

            di_zy10[i] = gaussian_DI(zlag, ylag, win)

        result = {}
        result['data_file'] = data_file
        result['lag'] = lag
        result['di_yz'] = di_yz
        result['di_zy'] = di_zy
        result['di_yz10'] = di_yz10
        result['di_zy10'] = di_zy10
        

Processing spikes


  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [16]:
dat = load_sabes(indy_files[0])

Processing spikes


100%|██████████| 1/1 [00:04<00:00,  4.74s/it]


In [18]:
dat_segmented = reach_segment_sabes(dat, start_time=0)

../../segmentation.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  m = straight[1]/straight[0]


In [36]:
y = [dat['spike_rates'][0, i1:i2] for i1, i2  in dat_segmented['transition_times']]
z = [dat['behavior'][i1:i2] for i1, i2, in dat_segmented['transition_times']]

In [40]:
gaussian_DI_trialized(y, z, 5)

1.9267784810566724

In [ ]:
``